In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def conv_layer(m_input,size_in,size_out,k_size_w,k_size_h,conv_stride,pool_k_size,pool_stride_size,name,num):
    with tf.name_scope(name+num):
        sdev = np.power(2.0/(k_size_w*k_size_h*size_in),0.5)
        print("sdev"+name+num+": ",sdev)
        w = tf.Variable(tf.truncated_normal([k_size_w,k_size_h,size_in,size_out],stddev = sdev),name=("w"+num))
        b = tf.Variable(tf.constant(.1,shape=[size_out]),name=("b"+num))

        conv = tf.nn.conv2d(m_input,w,strides=[1,conv_stride,conv_stride,1],padding="SAME")
        act = tf.nn.leaky_relu((conv+b),alpha=0.3)
        
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("act",act)
        return tf.nn.max_pool(act,ksize=[1,pool_k_size,pool_k_size,1],strides=[1,pool_stride_size,pool_stride_size,1],padding='SAME')


def fc_layer(m_input,size_in,size_out,name,num):
    with tf.name_scope(name+num):
        sdev = np.power(2.0/(size_in*size_out),0.5)
        print("sdev"+name+num+": ",sdev)
        w = tf.Variable(tf.truncated_normal([size_in,size_out],stddev = sdev),name=("w"+num))
        b = tf.Variable(tf.constant(0.1,shape=[size_out]),name=("b"+num))
        z = tf.matmul(m_input,w)
        act = tf.nn.leaky_relu(z+b,alpha=0.3)

        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("act",act)
        return act
    
def construct_weight_matricies():
    CW1 = tf.get_default_graph().get_tensor_by_name("conv1/w1:0")
    CW2 = tf.get_default_graph().get_tensor_by_name("conv2/w2:0")
    
    FW1 = tf.get_default_graph().get_tensor_by_name("FC1/w1:0")
    FW2 = tf.get_default_graph().get_tensor_by_name("FC2/w2:0")
    
    x = tf.get_default_graph().get_tensor_by_name("place_holder/x:0")
    y = tf.get_default_graph().get_tensor_by_name("place_holder/y:0")
        
    stk_CW1 = sess.run([tf.stack([CW1,CW1,CW1])],{x: np.random.rand(1,110,84,4),y: np.random.rand(1,4)})
    stk_CW2 = sess.run([tf.stack([CW2,CW2,CW2])],{x: np.random.rand(1,110,84,4),y: np.random.rand(1,4)})
    stk_FW1 = sess.run([tf.stack([FW1,FW1,FW1])],{x: np.random.rand(1,110,84,4),y: np.random.rand(1,4)})
    stk_FW2 = sess.run([tf.stack([FW2,FW2,FW2])],{x: np.random.rand(1,110,84,4),y: np.random.rand(1,4)})
    
    
    with tf.name_scope("constants"):
        tf.constant(np.squeeze(stk_CW1),name="C1w1_store")
        tf.constant(np.squeeze(stk_CW2),name="C2w2_store")
        tf.constant(np.squeeze(stk_FW1),name="FC1w1_store")
        tf.constant(np.squeeze(stk_FW2),name="FC2w2_store")
    return
    
        
    
def WB_tensors():
    tvar = tf.trainable_variables()    
    return tvar

def loss_fun():
    return

def iteration(sess,input_imgs):
    #0 Future, 1 Present, 2 Past
    
    return



In [3]:
def create_model(learning_rate,batch_size,conv_count,fc_count,conv_feats,fc_feats,conv_k_size,conv_stride):
    LOGDIR = r"C:\Users\Vishnu\Documents\EngProj\SSPlayer\log"
    if (len(conv_feats) != conv_count):
        return
    
    tf.reset_default_graph()
    
    with tf.name_scope("place_holder"):
        x = tf.placeholder(tf.float32,shape=[1,110,84,4],name="x")
        y = tf.placeholder(tf.float32,shape=[1,4],name="y")
            
    convs = []
    convs.append(x)
    conv_name="conv"
    conv_feats[0] = 4
    p = 0
    for i in range(0,conv_count-1):
        convs.append(conv_layer(convs[i],conv_feats[i],conv_feats[i+1],conv_k_size[p],conv_k_size[p],conv_stride[p],2,2,conv_name,str(i+1)))
    
    
    shape = (convs[conv_count-1]).get_shape().as_list()
    fc_feats[0] = shape[1]*shape[2]*conv_feats[conv_count-1]
    flatten = tf.reshape(convs[conv_count-1],[-1,fc_feats[0]])
    
    fcs = []
    fcs.append(flatten)
    fcs_name = "FC"
    for i in range(0,fc_count-1):
        fcs.append((fc_layer(fcs[i],fc_feats[i],fc_feats[i+1],fcs_name,str(i+1))))
    
    logts = fcs[len(fcs)-1]
    
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR)
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    construct_weight_matricies()
    #with tf.Session() as sess:
    #    sess.run(tf.global_variables_initializer())
    #    writer.add_graph(sess.graph)
    #    writer.close()
        
    #return convs,fcs,[x,y],summ,writer
    return sess,writer

In [4]:
conv_k_size = [8,4]
conv_stride = [4,2]
conv = [0,16,32]
fclyr = [0,256,4]
conv_count = len(conv)
fc_count = len(fclyr)
learning_rate = 1e-4
batch_size = 10
sess,writer = create_model(learning_rate,batch_size,conv_count,fc_count,conv,fclyr,conv_k_size,conv_stride)

sdevconv1:  0.08838834764831845
sdevconv2:  0.04419417382415922
sdevFC1:  0.0078125
sdevFC2:  0.04419417382415922


In [5]:
construct_weight_matricies()
writer.add_graph(sess.graph)

(3, 8, 8, 4, 16)
